In [2]:
import pyomo.environ as pyo
import numpy as np
import json
import pandas as pd
from omlt import OmltBlock, OffsetScaling
from omlt.neuralnet import ReluComplementarityFormulation, ReluPartitionFormulation
from omlt.io.keras import keras_reader
from tensorflow import keras
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds
import os
from sklearn.model_selection import train_test_split
import contextlib

In [4]:
df = pd.read_csv('dataframe')
inputs = ['FEED_BUT','FEED_PEN','FEED_HEX','DIST_RATE','Reflux Rate']
outputs = ['DIST_BUT','DIST_PEN','DIST_HEX',"Investment Cost",'Operating Cost']

X = df[inputs]
y = df[outputs]


mean_X = X.mean(axis=0)
std_X = X.std(axis=0)
print(std_X)
mean_y = y.mean(axis=0)
std_y = y.std(axis=0)
mean_y_weird = mean_y
std_y_weird = std_y
mean_X_weird = mean_X
std_X_weird = std_X

Xs = (X - mean_X) / std_X
ys = (y - mean_y) / std_y

X_train,X_test,y_train,y_test = train_test_split(Xs,ys, test_size=0.3, shuffle=True, random_state= 42)

X_train = np.asarray(X_train).astype('float64')
X_test = np.asarray(X_test).astype('float64')
y_train = np.asarray(y_train).astype('float64')
y_test = np.asarray(y_test).astype('float64')



x_offset, x_factor = X.mean().to_dict(), X.std().to_dict()
y_offset, y_factor = y.mean().to_dict(), y.std().to_dict()

scaled_lb = Xs.min()[inputs].values
scaled_ub = Xs.max()[inputs].values
scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )
scaler = scaler
input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}
print(len(df))
df_simple_csv = df[['FEED_BUT','FEED_PEN','FEED_HEX','DIST_RATE','Reflux Rate','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT','BOTT_PEN','BOTT_HEX',"Investment Cost","Operating Cost"]]
df_simple_csv.to_csv('nonsharp_dist_simple.csv',index=False) #uncomment to create dataframe thats fed into KKT-hPINN model

FEED_BUT       155.738601
FEED_PEN       173.252653
FEED_HEX       173.378937
DIST_RATE      217.643283
Reflux Rate    160.163593
dtype: float64
15003


In [19]:
feed_rate = 1000
mol_A = .33 #kmol/hr
mol_B = .33 #kmol/hr
mol_C = .34 #kmol/hr
def formulation(scaler_ab,scaler_bc,input_bounds_ab,input_bounds_bc,P1A,P1B,P1C):
    dist_rate_range = (.01,.99) #1% of column input stream to 99%
    
    P2A = feed_rate*mol_A - P1A
    P2B = feed_rate*mol_B - P1B
    P2C = feed_rate*mol_C - P1C
        
    model = pyo.ConcreteModel()

    #sets
    model.COMP = pyo.Set(initialize=['A','B','C']) #A is n-butene, B is n-pentene, C is n-hexane
    model.COLUMNS = pyo.Set(initialize=['A|BC','AB|C'])
    model.STREAMS = pyo.Set(initialize=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11',
                                   'F12','F13','F14','F15','F16','F17','F18','F19','F20','F21',
                                   'F22','F23']) #when referencing superstructure picture, subtract all streams by 1 so feed is 0 now and the last stream is 23
    model.PROD = pyo.Set(initialize=['P1','P2']) #products

    model.si = pyo.RangeSet(0,6) #index set to access splitters 2-8 in constraints
    model.ci = pyo.RangeSet(1,2) #index set to access columns 1-2
    model.fi = pyo.RangeSet(1,3) #index set to access streams connected to feed
    model.mci = pyo.RangeSet(0,1) #index set to access mixers 1 and 2
    model.mcp = pyo.RangeSet(0,1) #index set to access mixers 3 and 4
    #model.str_i = pyo.Set(initialize=[i for i in range(1, 23) if i not in [4,9]])
    model.str_i = pyo.RangeSet(1,23)
    inlet_split = [6,10,5,8,12,11,3] #indexes of streams that go into splitter to be called by model.f[i]
    outlet_split = [[1,2,3],[8,7],[13,12],[14,15],[16,17],[18,19],[20,21],[22,23]] #indexes of streams that go out of the splitter. First index corresponds to feed stream, not any of the streams in inlet_split list.
    inlet_column = [4,9] #inlet streams going into columns
    outlet_column = [[5,6],[10,11]]
    inlet_col_mix = [[1,13],[2,7]] #streams going into mixers before the column
    outlet_col_mix = [4,9] #streams going out of mixers before column
    inlet_prod_mix = [[14,16,18,20,22],[15,17,19,21,23]] #streams going out of mixers 3 and 4 going into products 1 and 2
    #data
    prod_comp = {
        ('P1','A'):P1A, ('P1','B'):P1B,('P1','C'):P1C,
        ('P2','A'):P2A, ('P2','B'):P2B,('P2','C'):P2C
    }

    #Parameters
    model.prod_comp = pyo.Param(model.PROD,model.COMP,initialize=prod_comp)

    #variables
    model.fa = pyo.VarList()
    for i in range(0,23):
        model.fa.add()
    for i in range(1, len(model.fa)+1):
        if i == 4 or i == 9:
            model.fa[i].setlb(0)
            model.fa[i].setub(1000) #can raise upperbound because of recycle stream
        else:
            model.fa[i].setlb(0)
            model.fa[i].setub(1000)
    model.fb = pyo.VarList()
    for i in range(0,23):
        model.fb.add()
    for i in range(1, len(model.fb)+1):
        if i == 4 or i == 9:
            model.fb[i].setlb(0)
            model.fb[i].setub(1000) #can raise upperbound because of recycle stream
        else:
            model.fb[i].setlb(0)
            model.fb[i].setub(1000)

    model.fc = pyo.VarList()
    for i in range(0,23):
        model.fc.add()
    for i in range(1, len(model.fc)+1):
        if i == 4 or i == 9:
            model.fc[i].setlb(0)
            model.fc[i].setub(1000) #can raise upperbound because of recycle stream
        else:
            model.fc[i].setlb(0)
            model.fc[i].setub(1000)
            

    model.ref_rate = pyo.VarList(domain=pyo.PositiveReals) #reflux ratio variable .75 -2
    model.dist_rate = pyo.VarList(initialize=0,bounds=(0,1000)) #distillate rate variable 0 - 1000
    model.cap_cost = pyo.VarList()#domain=pyo.PositiveReals) 
    model.op_cost = pyo.VarList()#domain=pyo.PositiveReals)

    for i in range(1,len(model.COLUMNS)+1):
        model.ref_rate.add()
        model.dist_rate.add()
        model.cap_cost.add()
        model.op_cost.add()

    #binary variables   
    model.y1 = pyo.Var(domain=pyo.Binary)
    model.y2 = pyo.Var(domain=pyo.Binary)

    #constraints
    #NN input variable constraints

    def dist_rate_constr1(model,i):
        ineq = model.dist_rate[i] >= dist_rate_range[0]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
        return ineq
    def dist_rate_constr2(model,i):
        ineq = model.dist_rate[i] <= dist_rate_range[1]*(model.fa[inlet_column[i-1]] + model.fb[inlet_column[i-1]] + model.fc[inlet_column[i-1]])
        return ineq
    

    def ref_rate_constr1(model,i):
        return model.ref_rate[i] >= .01*model.dist_rate[i]

    def ref_rate_constr2(model,i):
        return model.ref_rate[i] <= 2*model.dist_rate[i]


    #mass flow constraints
    #doing feed split constraints seperately because feed is constant so it's a slightly different form than the other split constraints
    def feed_splitA(model): #mass balance for first splitter
        ineq = mol_A*feed_rate == sum(model.fa[j] for j in outlet_split[0])
        return ineq
    def feed_mfA(model,i):
        ineq = mol_A*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fa[outlet_split[0][i-1]]
        return ineq

    def feed_splitB(model): #mass balance for first splitter
        ineq =  mol_B*feed_rate == sum(model.fb[j] for j in outlet_split[0])
        return ineq
    def feed_mfB(model,i):
        ineq = mol_B*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fb[outlet_split[0][i-1]]
        return ineq

    def feed_splitC(model): #making sure mole fracs are the same across streams 1 2 and 3
        ineq = mol_C*feed_rate == sum(model.fc[j] for j in outlet_split[0])
        return ineq
    def feed_mfC(model,i):
        ineq = mol_C*(model.fa[outlet_split[0][i-1]] + model.fb[outlet_split[0][i-1]] + model.fc[outlet_split[0][i-1]]) == model.fc[outlet_split[0][i-1]]
        return ineq

    #split constraints are making sure components flow in out streams add up to input component flow, and making sure mole ratios in out streams are same as mole ratios in in stream
    def split_flowA(model,i):
        ineq = model.fa[inlet_split[i]]== sum(model.fa[j] for j in outlet_split[i+1])
        return ineq
    def split_mfA(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fa[inlet_split[i]] \
        == model.fa[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    def split_flowB(model,i,j):
        ineq = model.fb[inlet_split[i]]== sum(model.fb[j] for j in outlet_split[i+1])
        return ineq
    def split_mfB(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fb[inlet_split[i]] \
        == model.fb[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    def split_flowC(model,i,j):
        ineq = model.fc[inlet_split[i]]== sum(model.fc[j] for j in outlet_split[i+1])
        return ineq
    def split_mfC(model,i,j):
        ineq = (model.fa[outlet_split[i+1][j-1]] +model.fb[outlet_split[i+1][j-1]] +model.fc[outlet_split[i+1][j-1]]) * model.fc[inlet_split[i]] \
        == model.fc[outlet_split[i+1][j-1]] * (model.fa[inlet_split[i]] + model.fb[inlet_split[i]] + model.fc[inlet_split[i]])
        return ineq

    #mixer constraints are making sure that component flow rates of in = out and making sure the amount mixed into the product streams is what we want.
    def mix_A_col(model,i):
        ineq = sum(model.fa[j]for j in inlet_col_mix[i]) == model.fa[outlet_col_mix[i]]
        #return (-10,sum(model.f[j]*model.xa[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xa[outlet_col_mix[i]],10)
        return ineq
    def mix_B_col(model,i):
        ineq = sum(model.fb[j] for j in inlet_col_mix[i]) == model.fb[outlet_col_mix[i]]
        #return (-10,sum(model.f[j]*model.xb[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xb[outlet_col_mix[i]],10)
        return ineq
    def mix_C_col(model,i):
        ineq = sum(model.fc[j]for j in inlet_col_mix[i]) == model.fc[outlet_col_mix[i]]
        #return (-10,sum(model.f[j]*model.xc[j] for j in inlet_col_mix[i]) - model.f[outlet_col_mix[i]]*model.xc[outlet_col_mix[i]],10)
        return ineq
    def mix_A_prod1(model):
        #ineq = sum(model.fa[j] for j in inlet_prod_mix[i]) == prod_comp['P1','A'] exact bounds
        return (1*prod_comp['P1','A'],sum(model.fa[j] for j in inlet_prod_mix[0]),prod_comp['P1','A']*1 + 1e-4) #loose bounds
    def mix_A_prod2(model):
        #ineq = sum(model.f[j]*model.xa[j] for j in inlet_prod_mix[i]) == prod_comp['P2','A']
        return (1*prod_comp['P2','A'],sum(model.fa[j] for j in inlet_prod_mix[1]),prod_comp['P2','A']*1 + 1e-4)
    def mix_B_prod1(model):
        #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P1','B']
        return (1*prod_comp['P1','B'],sum(model.fb[j] for j in inlet_prod_mix[0]),prod_comp['P1','B']*1 + 1e-4)
    def mix_B_prod2(model):
        #ineq = sum(model.fb[j] for j in inlet_prod_mix[i]) == prod_comp['P2','B']
        return (1*prod_comp['P2','B'],sum(model.fb[j] for j in inlet_prod_mix[1]),prod_comp['P2','B']*1 + 1e-4)
    def mix_C_prod1(model):
        #ineq = sum(model.fc[j] for j in inlet_prod_mix[i]) == prod_comp['P1','C']
        return (1*prod_comp['P1','C'],sum(model.fc[j] for j in inlet_prod_mix[0]),prod_comp['P1','C']*1 + 1e-4)
    def mix_C_prod2(model):
        #ineq = sum(model.fc[j] for j in inlet_prod_mix[i]) == prod_comp['P2','C']
        return (1*prod_comp['P2','C'],sum(model.fc[j] for j in inlet_prod_mix[1]),prod_comp['P2','C']*1+ 1e-4)

    #column constraints
    nn2_ab= keras.models.load_model('DIST simple model comp. 32 node.h5')
    nn2_bc= keras.models.load_model('DIST simple model comp. 32 node.h5')

    model.nn_ab = OmltBlock()
    model.nn_bc = OmltBlock()

    net_relu_ab = keras_reader.load_keras_sequential(nn2_ab,scaler,input_bounds)
    net_relu_bc = keras_reader.load_keras_sequential(nn2_bc,scaler,input_bounds)

    formulation_ab = ReluPartitionFormulation(net_relu_ab)
    formulation_bc = ReluPartitionFormulation(net_relu_bc)

    model.nn_ab.build_formulation(formulation_ab)
    model.nn_bc.build_formulation(formulation_bc)

    @model.Constraint()
    #column 1 inputs
    def col1_fa(mdl):
        return mdl.fa[4]  == mdl.nn_ab.inputs[0] * mdl.y1
    @model.Constraint()
    def col1_fb(mdl):
        return mdl.fb[4] == mdl.nn_ab.inputs[1] * mdl.y1
    @model.Constraint()
    def col1_fc(mdl):
        return  mdl.fc[4] == mdl.nn_ab.inputs[2] * mdl.y1
    @model.Constraint()
    def col1_distrate(mdl):
        return mdl.dist_rate[1] == mdl.nn_ab.inputs[3] * mdl.y1
    @model.Constraint()
    def col1_ref_rate(mdl):
        return mdl.ref_rate[1] == mdl.nn_ab.inputs[4] * mdl.y1
    #column 2 inputs
    @model.Constraint()
    def col2_fa(mdl):
        return mdl.fa[9] ==  mdl.nn_bc.inputs[0] * mdl.y2
    @model.Constraint()
    def col2_fb(mdl):
        return mdl.fb[9] == mdl.nn_bc.inputs[1] * mdl.y2
    @model.Constraint()
    def col2_fc(mdl):
        return mdl.fc[9] == mdl.nn_bc.inputs[2] * mdl.y2
    @model.Constraint()
    def col2_distrate(mdl):
        return mdl.dist_rate[2] == mdl.nn_bc.inputs[3] * mdl.y2
    @model.Constraint()
    def col2_ref_rate(mdl):
        return mdl.ref_rate[2] == mdl.nn_bc.inputs[4] * mdl.y2
    
    #columns 1 outputs
    @model.Constraint()
    def col1_da(mdl):
        return mdl.fa[5] == mdl.nn_ab.outputs[0] * mdl.y1
    @model.Constraint()
    def col1_db(mdl):
        return mdl.fb[5] == mdl.nn_ab.outputs[1] * mdl.y1
    @model.Constraint()
    def col1_dc(mdl):
        return mdl.fc[5] == mdl.nn_ab.outputs[2] * mdl.y1

    @model.Constraint()
    def col1_ba(mdl):
        return mdl.fa[6] == (mdl.fa[4] - mdl.fa[5]) * mdl.y1

    @model.Constraint()
    def col1_bb(mdl):
        return mdl.fb[6] == (mdl.fb[4] - mdl.fb[5]) * mdl.y1
    @model.Constraint()
    def col1_bc(mdl):
        return mdl.fc[6] == (mdl.fc[4] - mdl.fc[5]) * mdl.y1

    @model.Constraint()
    def col1_cap_cost(mdl):
        return mdl.cap_cost[1] == mdl.nn_ab.outputs[3]  * mdl.y1
    @model.Constraint()
    def col1_op_cost(mdl):
        return mdl.op_cost[1] == mdl.nn_ab.outputs[4] * mdl.y1

    #column 2 outputs
    @model.Constraint()
    def col2_da(mdl):
        return mdl.fa[10] == mdl.nn_bc.outputs[0] *mdl.y2
    @model.Constraint()
    def col2_db(mdl):
        return mdl.fb[10] == mdl.nn_bc.outputs[1] * mdl.y2
    @model.Constraint()
    def col2_dc(mdl):
        return mdl.fc[10] == mdl.nn_bc.outputs[2] * mdl.y2
    @model.Constraint()
    def col2_ba(mdl):
        return mdl.fa[11] == (mdl.fa[9] - mdl.fa[10]) * mdl.y2
    @model.Constraint()
    def col2_bb(mdl):
        return mdl.fb[11] == (mdl.fb[9] - mdl.fb[10]) * mdl.y2
    @model.Constraint()
    def col2_bc(mdl):
        return mdl.fc[11] == (mdl.fc[9] - mdl.fc[10]) * mdl.y2
    @model.Constraint()
    def col2_cap_cost(mdl):
        return mdl.cap_cost[2] == mdl.nn_bc.outputs[3]  * mdl.y2

    @model.Constraint()
    def col2_op_cost(mdl):
        return mdl.op_cost[2] == mdl.nn_bc.outputs[4] * mdl.y2

    model.dist_rate_constr1 = pyo.Constraint(model.ci,rule=dist_rate_constr1)
    model.dist_rate_constr2 = pyo.Constraint(model.ci,rule=dist_rate_constr2)
    
    model.ref_rate_constr1 = pyo.Constraint(model.ci,rule=ref_rate_constr1)
    model.ref_rate_constr2 = pyo.Constraint(model.ci,rule=ref_rate_constr2)

    #model.mole_frac_eq_constr = pyo.Constraint(model.str_i,rule=mole_frac_eq)

    #model.feed_splitflow_constr = pyo.Constraint(model.fi,rule=feed_splitflow)
    model.feed_splitA_constr = pyo.Constraint(rule=feed_splitA)
    model.feed_mfA_constr = pyo.Constraint(model.fi,rule=feed_mfA)
    model.feed_splitB_constr = pyo.Constraint(rule=feed_splitB)
    model.feed_mfB_constr = pyo.Constraint(model.fi,rule=feed_mfB)
    model.feed_splitC_constr = pyo.Constraint(rule=feed_splitC)
    model.feed_mfC_constr = pyo.Constraint(model.fi,rule=feed_mfC)

    model.splitA_constr = pyo.Constraint(model.si,rule=split_flowA)
    model.split_mfA_constr = pyo.Constraint(model.si,model.ci,rule=split_mfA)
    model.splitB_constr = pyo.Constraint(model.si,model.ci,rule=split_flowB)
    model.split_mfB_constr = pyo.Constraint(model.si,model.ci,rule=split_mfB)
    model.splitC_constr = pyo.Constraint(model.si,model.ci,rule=split_flowC)
    model.split_mfC_constr = pyo.Constraint(model.si,model.ci,rule=split_mfC)

    model.mix_A_col_constr = pyo.Constraint(model.mci,rule=mix_A_col)
    model.mix_B_col_constr = pyo.Constraint(model.mci,rule=mix_B_col)
    model.mix_C_col_constr = pyo.Constraint(model.mci,rule=mix_C_col)


    model.mix_A_prod1_constr = pyo.Constraint(rule=mix_A_prod1)
    model.mix_A_prod2_constr = pyo.Constraint(rule=mix_A_prod2)
    model.mix_B_prod1_constr = pyo.Constraint(rule=mix_B_prod1)
    model.mix_B_prod2_constr = pyo.Constraint(rule=mix_B_prod2)
    model.mix_C_prod1_constr = pyo.Constraint(rule=mix_C_prod1)
    model.mix_C_prod2_constr = pyo.Constraint(rule=mix_C_prod2)

    #Objective function
    model.obj = pyo.Objective(expr=(model.cap_cost[1] + model.cap_cost[2] + model.op_cost[1] + model.op_cost[2]),sense=pyo.minimize)

    return model

def validation_save(model,P1A,P1B,P1C):
    feed_a1 = pyo.value(model.fa[4])
    feed_b1 = pyo.value(model.fb[4])
    feed_c1 = pyo.value(model.fc[4])
    dist_a1 = pyo.value(model.fa[5])
    dist_b1 = pyo.value(model.fb[5])
    dist_c1 = pyo.value(model.fc[5])
    bott_a1 = pyo.value(model.fa[6])
    bott_b1 = pyo.value(model.fb[6])
    bott_c1 = pyo.value(model.fc[6])
    ref_rate1 = pyo.value(model.ref_rate[1])
    dist_rat1 = pyo.value(model.dist_rate[1])
    cap_cost1 = pyo.value(model.cap_cost[1])*1e4
    op_cost1 = pyo.value(model.op_cost[1])*1e4
    col1_vals = [feed_a1,feed_b1,feed_c1,dist_a1,dist_b1,dist_c1,bott_a1,bott_b1,bott_c1,ref_rate1,dist_rat1,cap_cost1,op_cost1]

    feed_a2 = pyo.value(model.fa[9])
    feed_b2 = pyo.value(model.fb[9])
    feed_c2 = pyo.value(model.fc[9])
    dist_a2 = pyo.value(model.fa[10])
    dist_b2 = pyo.value(model.fb[10])
    dist_c2 = pyo.value(model.fc[10])
    bott_a2 = pyo.value(model.fa[11])
    bott_b2 = pyo.value(model.fb[11])
    bott_c2 = pyo.value(model.fc[11])
    ref_rate2 = pyo.value(model.ref_rate[2])
    dist_rat2 = pyo.value(model.dist_rate[2])
    cap_cost2 = pyo.value(model.cap_cost[2])*1e4
    op_cost2 = pyo.value(model.op_cost[2])*1e4
    col2_vals = [feed_a2,feed_b2,feed_c2,dist_a2,dist_b2,dist_c2,bott_a2,bott_b2,bott_c2,ref_rate2,dist_rat2,cap_cost2,op_cost2]

    df_save = pd.DataFrame([col1_vals,col2_vals], columns=['FEED_BUT','FEED_PEN','FEED_HEX','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT',
                                                 'BOTT_PEN','BOTT_HEX','REF_RATE','DIST_RATE','CAP_COST','OP_COST'])
    df_save.to_csv('nn_val_values/{}-{}-{} validation values'.format(P1A,P1B,P1C))
                       
    return model

def save_output_to_file(file,model,P1A,P1B,P1C):
    with open(file, 'w') as f:
        with contextlib.redirect_stdout(f):
            print("Product 1 A flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fa[14]) + pyo.value(model.fa[16])
                                                                + pyo.value(model.fa[18]) + pyo.value(model.fa[20])
                                                                + pyo.value(model.fa[22]),P1A))

            print("Product 2 A flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fa[15]) + pyo.value(model.fa[17])
                                                                + pyo.value(model.fa[19]) + pyo.value(model.fa[21])
                                                                + pyo.value(model.fa[23]),feed_rate*mol_A - P1A))

            print("Product 1 B flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fb[14]) + pyo.value(model.fb[16])
                                                                + pyo.value(model.fb[18]) + pyo.value(model.fb[20])
                                                                + pyo.value(model.fb[22]),P1B))

            print("Product 2 B flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fb[15]) + pyo.value(model.fb[17])
                                                                + pyo.value(model.fb[19]) + pyo.value(model.fb[21])
                                                                + pyo.value(model.fb[23]),feed_rate*mol_B - P1B))
            print("Product 1 C flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fc[14]) + pyo.value(model.fc[16])
                                                                + pyo.value(model.fc[18]) + pyo.value(model.fc[20])
                                                                + pyo.value(model.fc[22]),P1C))

            print("Product 2 C flowrate is {:.3f} out of {:.3f}".format(pyo.value(model.fc[15]) + pyo.value(model.fc[17])
                                                                + pyo.value(model.fc[19]) + pyo.value(model.fc[21])
                                                                + pyo.value(model.fc[23]),feed_rate*mol_C - P1C))

            print("Column 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[4]) - pyo.value(model.fa[5])
                                                                                             - pyo.value(model.fa[6]), pyo.value(model.fb[4])
                                                                                             - pyo.value(model.fb[5]) - pyo.value(model.fb[6]),
                                                                                             pyo.value(model.fc[4]) - pyo.value(model.fc[5])
                                                                                             - pyo.value(model.fc[6])))
            print("Column 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[9]) - pyo.value(model.fa[10])
                                                                                             - pyo.value(model.fa[11]), pyo.value(model.fb[9])
                                                                                             - pyo.value(model.fb[10]) - pyo.value(model.fb[11]),
                                                                                             pyo.value(model.fc[9]) - pyo.value(model.fc[10])
                                                                                             - pyo.value(model.fc[11])))
            print("Mixer 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[4]) - pyo.value(model.fa[13])
                                                                                             - pyo.value(model.fa[1]), pyo.value(model.fb[4])
                                                                                             - pyo.value(model.fb[13]) - pyo.value(model.fb[1]),
                                                                                             pyo.value(model.fc[4]) - pyo.value(model.fc[13])
                                                                                             - pyo.value(model.fc[1])))
            print("Mixer 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[9]) - pyo.value(model.fa[7])
                                                                                             - pyo.value(model.fa[2]), pyo.value(model.fb[9])
                                                                                             - pyo.value(model.fb[7]) - pyo.value(model.fb[2]),
                                                                                             pyo.value(model.fc[9]) - pyo.value(model.fc[7])
                                                                                             - pyo.value(model.fc[2])))
            print("Splitter 1 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(mol_A*feed_rate - pyo.value(model.fa[1])
                                                                                             - pyo.value(model.fa[2]) - pyo.value(model.fa[3]), mol_B*feed_rate
                                                                                             - pyo.value(model.fb[1]) - pyo.value(model.fb[2]) - pyo.value(model.fb[3]),
                                                                                             mol_C*feed_rate - pyo.value(model.fc[1])
                                                                                             - pyo.value(model.fc[2]) - pyo.value(model.fc[3])))
            print("Splitter 2 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[6]) - pyo.value(model.fa[7])
                                                                                             - pyo.value(model.fa[8]), pyo.value(model.fb[6])
                                                                                             - pyo.value(model.fb[7]) - pyo.value(model.fb[8]),
                                                                                             pyo.value(model.fc[6]) - pyo.value(model.fc[7])
                                                                                             - pyo.value(model.fc[8])))
            print("Splitter 3 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[10]) - pyo.value(model.fa[13])
                                                                                             - pyo.value(model.fa[12]), pyo.value(model.fb[10])
                                                                                             - pyo.value(model.fb[13]) - pyo.value(model.fb[12]),
                                                                                             pyo.value(model.fc[10]) - pyo.value(model.fc[13])
                                                                                             - pyo.value(model.fc[12])))
            print("Splitter 4 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[5]) - pyo.value(model.fa[14])
                                                                                             - pyo.value(model.fa[15]), pyo.value(model.fb[5])
                                                                                             - pyo.value(model.fb[14]) - pyo.value(model.fb[15]),
                                                                                             pyo.value(model.fc[5]) - pyo.value(model.fc[14])
                                                                                             - pyo.value(model.fc[15])))
            print("Splitter 5 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[8]) - pyo.value(model.fa[16])
                                                                                             - pyo.value(model.fa[17]), pyo.value(model.fb[8])
                                                                                             - pyo.value(model.fb[16]) - pyo.value(model.fb[17]),
                                                                                             pyo.value(model.fc[8]) - pyo.value(model.fc[16])
                                                                                             - pyo.value(model.fc[17])))
            print("Splitter 6 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[12]) - pyo.value(model.fa[18])
                                                                                             - pyo.value(model.fa[19]), pyo.value(model.fb[12])
                                                                                             - pyo.value(model.fb[18]) - pyo.value(model.fb[19]),
                                                                                             pyo.value(model.fc[12]) - pyo.value(model.fc[18])
                                                                                             - pyo.value(model.fc[19])))
            print("Splitter 7 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[11]) - pyo.value(model.fa[20])
                                                                                             - pyo.value(model.fa[21]), pyo.value(model.fb[11])
                                                                                             - pyo.value(model.fb[20]) - pyo.value(model.fb[21]),
                                                                                             pyo.value(model.fc[11]) - pyo.value(model.fc[20])
                                                                                             - pyo.value(model.fc[21])))
            print("Splitter 8 violation is {:.3f} for A, {:.3f} for B, and {:.3f} for C".format(pyo.value(model.fa[3]) - pyo.value(model.fa[23])
                                                                                             - pyo.value(model.fa[22]), pyo.value(model.fb[3])
                                                                                             - pyo.value(model.fb[23]) - pyo.value(model.fb[22]),
                                                                                             pyo.value(model.fc[3]) - pyo.value(model.fc[23])
                                                                                             - pyo.value(model.fc[22])))
            print("\n \n")
            print("Col 1 capital cost is ${:.2f}".format(pyo.value(model.cap_cost[1])*1e4))
            print("Col 2 capital cost is ${:.2f}".format(pyo.value(model.cap_cost[2])*1e4))
            print("Col 1 operating cost is ${:.2f}".format(pyo.value(model.op_cost[1])*1e4))
            print("Col 2 operating cost is ${:.2f}".format(pyo.value(model.op_cost[2])*1e4))
            print("\n")
           
            print("Col 1 reflux rate is {:.3f}".format(pyo.value(model.ref_rate[1])))
            print("Col 1 dist rate is {:.3f}".format(pyo.value(model.dist_rate[1])))
           
            print("\n")
            
            print("Col 2 reflux rate is {:.3f}".format(pyo.value(model.ref_rate[2])))
            print("Col 2 dist rate is {:.3f}".format(pyo.value(model.dist_rate[2])))
            
            print("\n")
            for i in range(1,24):
                print("Stream {} A flowrate is {:.3f}".format(i,pyo.value(model.fa[i])))
                print("Stream {} B flowrate is {:.3f}".format(i,pyo.value(model.fb[i])))
                print("Stream {} C flowrate is {:.3f}".format(i,pyo.value(model.fc[i])))

In [20]:
save_file = True #save the log file of the optimization
Products = [[165,165,170]]

for prod in Products:
    P1A = prod[0]    
    P1B = prod[1]
    P1C = prod[2]
    if save_file == True:
        model = formulation(scaler,scaler,input_bounds,input_bounds,P1A,P1B,P1C)
        print("\n")
        print("solving for {} {} {}".format(P1A,P1B,P1C))
        print("\n")
        solver = pyo.SolverFactory('gurobi')
        solver.options["LogFile"] = 'nn_log_files/{}-{}-{}'.format(P1A,P1B,P1C)
        status = solver.solve(model, tee=True)# options={'TimeLimit': 3600}
    else:    
        model = formulation(scaler,scaler,input_bounds,input_bounds,P1A,P1B,P1C)
        solver = pyo.SolverFactory('gurobi')
        status = solver.solve(model, tee=True,options={'TimeLimit': 3600})# options={'TimeLimit': 3600}
        
    validation_save(model,P1A,P1B,P1C)
    save_output_to_file('nn_output_values/{}-{}-{}'.format(P1A,P1B,P1C)  + " output values",model,P1A,P1B,P1C)



solving for 165 165 170


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-29
Read LP format model from file C:\Users\RazzyZac\AppData\Local\Temp\tmpvgv3nide.pyomo.lp
Reading time = 0.06 seconds
x1: 857 rows, 425 columns, 3377 nonzeros
Set parameter LogFile to value "nn_log_files/165-165-170"
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 857 rows, 425 columns and 3377 nonzeros
Model fingerprint: 0xcfda4630
Model has 68 quadratic constraints
Variable types: 359 continuous, 66 integer (66 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [9e-04, 2e+03]
  RHS range        [9e-04, 3e+02]
Presolve

ApplicationError: Solver (gurobi) did not exit normally